# 案例: 高保真设备仿真 (数据生成)

本脚本使用高保真的`NetworkSolver`来模拟一个渠道系统，扮演“真实世界”或“数字孪生体”的角色。它的主要目的是基于执行器的输入（如水闸开度变化）来生成传感器数据。

生成的数据将被保存到CSV文件中，以供后续的系统辨识等任务使用。

### 系统拓扑:
- 一个使用圣维南方程（St. Venant equations）建模的主要渠道。
- 在渠道末端有一个可控水闸。
- 一个上游边界条件，提供恒定的入流。
- 一个下游边界条件，代表固定的水位。

### 1. 导入库并加载配置

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from swp.simulation_identification.physical_objects.st_venant_reach import StVenantReach
from swp.simulation_identification.hydro_nodes.gate_node import GateNode
from swp.core_engine.solver.network_solver import NetworkSolver

CONFIG_PATH = 'examples/plant_simulation.json'
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

### 2. 定义系统搭建和运行函数

In [ ]:
def setup_plant(cfg):
    """根据配置搭建高保真物理网络。"""
    print("\n--- 搭建高保真物理网络 ---")
    sim_cfg = cfg['simulation']
    ic_cfg = cfg['initial_conditions']
    comp_cfgs = cfg['components']

    # 上游主渠道
    ur_cfg = comp_cfgs['upstream_reach']
    upstream_reach = StVenantReach(
        name="upstream_reach",
        initial_H=np.full(ur_cfg['num_points'], ic_cfg['depth']),
        initial_Q=np.full(ur_cfg['num_points'], ic_cfg['flow']),
        **ur_cfg
    )

    # 下游虚拟渠道
    dr_cfg = comp_cfgs['downstream_reach']
    downstream_reach = StVenantReach(
        name="downstream_reach",
        initial_H=np.full(dr_cfg['num_points'], ic_cfg['depth'] + dr_cfg['initial_depth_offset']),
        initial_Q=np.full(dr_cfg['num_points'], ic_cfg['flow']),
        **dr_cfg
    )

    # 控制闸门节点
    gate_cfg = comp_cfgs['control_gate']
    control_gate = GateNode(name="control_gate", **gate_cfg)
    control_gate.set_opening(gate_cfg['initial_opening'])

    # 创建并配置求解器
    solver = NetworkSolver(dt=sim_cfg['dt'], theta=sim_cfg['theta'])
    solver.add_component(upstream_reach)
    solver.add_component(downstream_reach)
    solver.add_component(control_gate)
    control_gate.link_to_reaches(up_obj=upstream_reach, down_obj=downstream_reach)
    control_gate.upstream_idx = -1
    control_gate.downstream_idx = 0

    # 设置边界条件
    solver.add_boundary_condition(upstream_reach, 'Q', 0, lambda t: ic_cfg['flow'])
    solver.add_boundary_condition(downstream_reach, 'H', -1, lambda t: ic_cfg['depth'] + dr_cfg['initial_depth_offset'])
    
    print("物理网络搭建成功。")
    return solver, upstream_reach, downstream_reach, control_gate

def execute_and_log(solver, up_reach, down_reach, gate, cfg):
    """运行仿真并记录数据。"""
    sim_cfg = cfg['simulation']
    num_steps = sim_cfg['num_steps']
    events = {event['time_step']: event for event in cfg['control_events']}
    data_log = []

    print("\n--- 开始仿真和数据记录 ---")
    for i in range(num_steps):
        current_time = i * sim_cfg['dt']
        if i in events:
            event = events[i]
            print(f"\n!!! 时间 {current_time}s: 对水闸施加阶跃输入。 !!!\n")
            if event['action'] == 'set_opening':
                gate.set_opening(event['value'])
        
        log_entry = {
            "timestamp": current_time,
            "gate_opening": gate.opening,
            "upstream_water_level": up_reach.H[-1],
            "downstream_water_level": down_reach.H[0]
        }
        data_log.append(log_entry)
        solver.step(current_time)
        if i % 10 == 0:
            print(f"  步 {i}/{num_steps}: 闸门开度={gate.opening:.2f}, 上游水位={up_reach.H[-1]:.3f}m, 下游水位={down_reach.H[0]:.3f}m")
    
    print("\n--- 仿真结束 ---")
    return pd.DataFrame(data_log)

### 3. 运行仿真并保存数据

In [ ]:
solver, up_reach, down_reach, gate = setup_plant(config)
results_df = execute_and_log(solver, up_reach, down_reach, gate, config)

output_filename = config['simulation']['output_filename']
results_df.to_csv(output_filename, index=False)
print(f"\n--- 设备仿真数据已保存至 {output_filename} ---")
print("数据预览:")
print(results_df.head())

### 4. 可视化生成的传感器数据

In [ ]:
fig, ax1 = plt.subplots(figsize=(15, 7))
fig.suptitle('生成的设备传感器数据', fontsize=16)

ax1.plot(results_df['timestamp'], results_df['upstream_water_level'], 'b-', label='上游水位 (传感器1)')
ax1.plot(results_df['timestamp'], results_df['downstream_water_level'], 'c-', label='下游水位 (传感器2)')
ax1.set_xlabel('时间 (s)')
ax1.set_ylabel('水位 (m)', color='b')
ax1.tick_params(axis='y', labelcolor='b')
ax1.grid(True)

ax2 = ax1.twinx()
ax2.plot(results_df['timestamp'], results_df['gate_opening'], 'r--', label='水闸开度 (执行器)')
ax2.set_ylabel('开度', color='r')
ax2.tick_params(axis='y', labelcolor='r')

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper right')

plt.show()